# 3 table join (xapm_db_stat, xapm_db_os_stat, xapm_db_wait)

by pdm

In [2]:
from __future__ import division
import google.datalab.bigquery as bq
import matplotlib.pyplot as plot
import numpy as np
import pandas as pd

# 테이블 3개 db_id = 6 인놈에 대해서만, time을 변수로 조인하기

In [2]:
%bq tables list --project exem-191100 --dataset ods_im5

In [27]:
%%bq query --name join
SELECT t1_os.cpu_usage, t1_os.max_cpu_usage, t1_os.free_memory, t1_os.min_free_memory, 
  t1_os.total_memory, t2_st.stat_id, t2_st.value, t3_wa.event_id, t3_wa.wait_count, t3_wa.wait_time, t3_wa.wait_timeout, t1_os.time
FROM `exem-191100.ods_im5.xapm_db_os_stat` as t1_os, `exem-191100.ods_im5.xapm_db_stat` as t2_st, `exem-191100.ods_im5.xapm_db_wait` as t3_wa
WHERE t1_os.db_id = 6 and t2_st.db_id = 6 and t3_wa.db_id = 6 and t1_os.time = t2_st.time and t2_st.time = t3_wa.time
ORDER BY t1_os.time, stat_id, event_id

In [28]:
%bq sample --query join --count 1000

cpu_usage,max_cpu_usage,free_memory,min_free_memory,total_memory,stat_id,value,event_id,wait_count,wait_time,wait_timeout,time
5244,6670,246592,241860,6117496,0,0,1,1,300,1,2017-11-03 15:00:00
5244,6670,246592,241860,6117496,0,0,7,14,3286,14,2017-11-03 15:00:00
5244,6670,246592,241860,6117496,0,0,11,0,0,0,2017-11-03 15:00:00
5244,6670,246592,241860,6117496,0,0,14,0,0,0,2017-11-03 15:00:00
5244,6670,246592,241860,6117496,0,0,15,0,0,0,2017-11-03 15:00:00
5244,6670,246592,241860,6117496,0,0,20,0,0,0,2017-11-03 15:00:00
5244,6670,246592,241860,6117496,0,0,22,0,0,0,2017-11-03 15:00:00
5244,6670,246592,241860,6117496,0,0,65,0,0,0,2017-11-03 15:00:00
5244,6670,246592,241860,6117496,0,0,68,0,0,0,2017-11-03 15:00:00
5244,6670,246592,241860,6117496,0,0,71,0,0,0,2017-11-03 15:00:00


# join table은 유용할 것 같으므로 프로젝트에 저장을 시켜놓자

- 하지만 실패했다.
- allotted memory를 초과한다.. **좋은방법을 아시는 분..?**

In [23]:
%bq execute --query join --table exem-191100.ods_im5.join_xapm_db_stat1_osstat2_wait3 --large


HTTP request failed: Resources exceeded during query execution: The query could not be executed in the allotted memory. ORDER BY operator used too much memory.

In [3]:
join_sql = 'SELECT t1_os.cpu_usage, t1_os.max_cpu_usage, t1_os.free_memory, t1_os.min_free_memory, t1_os.total_memory, t2_st.stat_id, t2_st.value, t3_wa.event_id, t3_wa.wait_count, t3_wa.wait_time, t3_wa.wait_timeout, t1_os.time FROM `exem-191100.ods_im5.xapm_db_os_stat` as t1_os, `exem-191100.ods_im5.xapm_db_stat` as t2_st, `exem-191100.ods_im5.xapm_db_wait` as t3_wa WHERE t1_os.db_id = 6 and t2_st.db_id = 6 and t3_wa.db_id = 6 and t1_os.time = t2_st.time and t2_st.time = t3_wa.time ORDER BY t1_os.time, stat_id, event_id'

query = bq.Query(join_sql)

join_df = query.execute(output_options = bq.QueryOutput.dataframe()).result()

RequestException: HTTP request failed: Resources exceeded during query execution: The query could not be executed in the allotted memory. ORDER BY operator used too much memory.

**이것 또한 양이 많아서 메모리를 초과한다..!**